In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import time
import json
import pandas as pd

from selenium.webdriver.common.by import By

from method.ours.utils import create_driver, get_xpath

In [3]:
HEADLESS = False
APP_NAME = 'Air Canada'
APP_URL = 'https://www.aircanada.com/ca/en/aco/home.html'

# Go to the Page

In [4]:
driver = create_driver(HEADLESS)
driver.get(APP_URL)

# Find All the Forms

In [5]:
forms = driver.find_elements(By.TAG_NAME, 'form')

In [7]:
form_xpaths = []

for i, form in enumerate(forms):
    xpath = get_xpath(driver, form)
    form_xpaths.append(xpath)
    
    print(f'Form {i} with XPATH:')
    print(xpath)
    
    labels = '\n'.join(map(lambda x: x.text, form.find_elements(By.TAG_NAME, 'label')))
    print('Form content:')
    print(labels)
    print()

Form 0 with XPATH:
//BODY/AC-WEB-APP[1]/DIV[1]/DIV[1]/DIV[1]/NGX-AC-HEADER[1]/NGX-AC-HEADER-PRES[1]/HEADER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[3]/NGX-AC-SITE-SEARCH[1]/FORM[1]
Form content:
Search

Form 1 with XPATH:
//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]
Form content:
Round-trip
One-way
Multi-city/Stopover
Book with points
From
To
Departure
Return



In [8]:
selected_form_index = int(input('Choose one of the forms to interact with: '))

Choose one of the forms to interact with.1


In [11]:
selected_form = forms[selected_form_index]
selected_form_xpath = form_xpaths[selected_form_index]

# Find Passing Values

<span style="color: red;">INSTRUCTION</span>: Use the form to find a successful combination of input values.

# Input Passing Values

In [17]:
driver.get(APP_URL)

<span style="color: red;">INSTRUCTION</span>: Do the necessary interactions to get back to the intended form.

In [18]:
form = driver.find_elements(By.TAG_NAME, 'form')[selected_form_index]

In [37]:
inputs = form.find_elements(
    By.XPATH,
    f'{selected_form_xpath}//input | {selected_form_xpath}//textarea | {selected_form_xpath}//select'
)
inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [39]:
passing_values = {}

for element in inputs:
    element_id = element.get_attribute('id') or get_xpath(driver, element)
    print(element.get_attribute('outerHTML'))
    value = input('Enter a successful value for this input: ')
    passing_values[element_id] = value

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_R" name="tripType" value="R" aria-disabled="false" aria-invalid="false" aria-checked="true">
Enter a successful value for this input: True
<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_O" name="tripType" value="O" aria-disabled="false" aria-invalid="false" aria-checked="false">
Enter a successful value for this input: False
<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_M" name="tripType" value="M" aria-disabled="false" aria-invalid="false" aria-checked="false">
Enter a successful value for this input: False
<input type="checkbox" role="checkbox" class="abc-form-element-main abc-form-element-checkbox ng-untouched ng-pristine ng-valid" id="bkmgF

In [40]:
passing_values

{'bkmgFlights_tripTypeSelector_R': 'True',
 'bkmgFlights_tripTypeSelector_O': 'False',
 'bkmgFlights_tripTypeSelector_M': 'False',
 'bkmgFlights_searchTypeToggle': 'False',
 'bkmgFlights_origin_trip_1': 'Vancouver YVR',
 'bkmgFlights_destination_trip_1': 'Toronto YYZ',
 'bkmgFlights_travelDates_1-formfield-1': '30/07/2023',
 'bkmgFlights_travelDates_1-formfield-2': '28/08/2023'}

# Input Failing Values

In [ ]:
driver.get(APP_URL)

<span style="color: red;">INSTRUCTION</span>: Do the necessary interactions to get back to the intended form.

In [ ]:
form = driver.find_elements(By.TAG_NAME, 'form')[selected_form_index]

In [ ]:
inputs = form.find_elements(
    By.XPATH,
    f'{selected_form_xpath}//input | {selected_form_xpath}//textarea | {selected_form_xpath}//select'
)
inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [ ]:
failing_values = {}


for element in inputs:
    element_id = element.get_attribute('id') or get_xpath(driver, element)
    print(element.get_attribute('outerHTML'))
    value = input('Enter a failing value for this input: ')
    failing_values[element_id] = value

# Save the results

In [41]:
PATH_TO_DATASET = './annotations.csv'

In [50]:
df = pd.read_csv(PATH_TO_DATASET)

In [51]:
new_row = [
    APP_NAME,
    APP_URL,
    selected_form_xpath,
    json.dumps(passing_values),
    'SUCCESS',
    None
]

In [52]:
df.loc[len(df)] = new_row

In [53]:
df

,application,url,form_id,values,success_fail,submission_state
0,Air Canada,https://www.aircanada.com/ca/en/aco/home.html,//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-...,"{""bkmgFlights_tripTypeSelector_R"": ""True"", ""bk...",SUCCESS,None


In [54]:
df.to_csv(PATH_TO_DATASET, index=False)

requirements:
- [x] Go to a page
- [x] Show all of forms and then select the one that we want to use
- [x] Save the identification of the form (application + url + id/xpath)
- [x] Let user try the intended form
- [x] After trying the form, iterate over the inputs one by one, ask for a passing value for each one.
- [x] Save the passing values.
- [] Iterates over the inputs once again, prompt a value from fuzz-samples, try changing that input field with that value.
- [] Save the resulting state of the page if user decides its a new state.
- [x] Let user try other values that might fail.
- [] Saves the resulting state of the page if user decides its a new state.
- [] Aggregate the states.
- [x] User has to enter the states into a specific document format.